
Copyright (C) 2017 The HDF Group  
All rights reserved

This example code illustrates how to access and visualize a GES DISC ACOS
Swath HDF5 file in Python.

If you have any questions, suggestions, or comments on this example, please use
the HDF-EOS Forum (http://hdfeos.org/forums).  If you would like to see an
example of any other NASA HDF/HDF-EOS data product that is not listed in the
HDF-EOS Comprehensive Examples page (http://hdfeos.org/zoo), feel free to
contact us at eoshelp@hdfgroup.org or post it at the HDF-EOS Forum
(http://hdfeos.org/forums).

# Access ACOS data from GES DISC via OPeNDAP

 If you try to access ACOS OPeNDAP data with Panoply or IDV, you will get an error message *URI Too Large* like below. 
 
 ![png](acos_L2s_panoply_error.png)
 
 This example will illustrate how to subset and download data from ACOS OPeNDAP data and save it as netCDF4. Then, we will augment the downloaded data so that visualization tool can understand and visualize data as shown below.
 
 ![png](acos_L2s_IDV_animation.gif)
 
 The first step is to make sure that your NASA Earthdata Login username and password works with OPeNDAP server.

